<a href="https://colab.research.google.com/github/giselesousar/msr_tests/blob/main/test_treemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
# Install and update plotly
!pip install plotly==5.5.0

In [3]:
# https://github.com/plotly/plotly.py
import plotly.express as px

In [13]:
import os
from pathlib import Path
import plotly.graph_objects as go

# Clone repository

In [16]:
![ -d promocity ] && echo "Remove diretório promocity antigo" && rm -R promocity
!git clone https://github.com/armandossrecife/promocity.git

Cloning into 'promocity'...
remote: Enumerating objects: 841, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 841 (delta 10), reused 21 (delta 2), pack-reused 798
Receiving objects: 100% (841/841), 4.00 MiB | 2.59 MiB/s, done.
Resolving deltas: 100% (324/324), done.


In [7]:
#For google colab
!sudo apt install tree

Unable to locate an executable at "/Library/Java/JavaVirtualMachines/jdk-11.0.11.jdk/Contents/Home/bin/apt" (-1)


In [31]:
# Cria um arquivo contendo a quantidade de LOC por arquivo
!find promocity -name *.java | xargs wc -l > locarquivosjava.txt
list_locs_of_files = !cat locarquivosjava.txt

# Cria uma lista com elementos que representam o LOC e o arquivo
# (Loc, arquivo)
list_locs_of_files_updated = []
for each in list_locs_of_files:
  elementos = each.split(' ') 
  item = elementos[-2], elementos[-1]
  list_locs_of_files_updated.append(item)

In [32]:
def search_loc_of_file(file_name, list):
  for each in list:
    if file_name in each[1]:
      return int(each[0])

In [33]:
search_loc_of_file('UserController.java', list_locs_of_files_updated)

587

# Analyzer

In [34]:
# Lista todos os arquivos e diretorios
list_of_files_and_directories = !cd promocity && tree -i -f

In [35]:
# Substitui o . por promocity/
list_of_files_and_directories_updated = [each.replace('./', 'promocity/') for each in list_of_files_and_directories]

In [36]:
# Escolhe o diretorio do source java
# Lista apenas arquivos e diretorios do src/main/java
list_of_files_and_directories_src = []

for item in list_of_files_and_directories_updated:
  if 'src' in item:
    if 'src/main/java/' in item:
      list_of_files_and_directories_src.append(item)

In [37]:
# popula os labels, parents e values
# adaptado de https://plotly.com/python/treemaps/
labels = []
parents = []
for each in list_of_files_and_directories_src:
  nome_item = each.split('/')[-1]
  labels.append(nome_item)
  if os.path.isdir(each):
      pai = os.path.dirname(each)
      pai = pai.split('/')[-1]
      if pai == 'java':
        pai = ' '
      parents.append(pai)
  else: 
    elemento = Path(each).parent
    elemento = str(elemento)
    elemento = elemento.split('/')[-1]
    parents.append( str(elemento) )

In [38]:
values = []
for i in range(0, len(labels) ):
  each = list_of_files_and_directories_src[i]
  nome_item = each.split('/')[-1]
  loc = search_loc_of_file(nome_item, list_locs_of_files_updated)
  if '.java' in nome_item: 
    values.append(loc)
  else:
    values.append(1)

In [39]:
fig = go.Figure(go.Treemap(
    labels = labels,
    values = values,
    parents = parents,
    marker_colorscale = 'Blues'
))

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))

fig.show()